Universidad del Valle de Guatemala <br>
Security Data Science <br>
Jorge Yass <br>
Douglas de León - 18037 <br>
Augusto Alonso -  <br>

# Laboratorio 6 - Hunting with Data Science

## Parte 1 - Filtrado y preprocesamiento

In [16]:
import json
import pickle

import pandas as pd
import numpy as np

In [2]:
data = []
with open("large_eve.json") as file:
    for line in file.readlines():
        data.append(json.loads(line))

In [3]:
print("Cantidad de registros:", "{:,}".format(len(data)))

Cantidad de registros: 746,909


In [4]:
only_dns = list(filter(lambda x: "dns" in x, data))

In [5]:
print("Cantidad de registros DNS: ", "{:,}".format(len(only_dns)))

Cantidad de registros DNS:  15,749


In [6]:
print("Ejemplos: \n")
for ej in only_dns[:2]:
    print(json.dumps(ej, indent=4))

Ejemplos: 

{
    "timestamp": "2017-07-22T17:33:16.661646-0500",
    "flow_id": 1327836194150542,
    "pcap_cnt": 22269,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 59680,
    "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 15529,
        "rrname": "api.wunderground.com",
        "rrtype": "A",
        "tx_id": 0
    }
}
{
    "timestamp": "2017-07-22T17:33:24.990320-0500",
    "flow_id": 2022925111925872,
    "pcap_cnt": 54352,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 38051,
    "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 58278,
        "rrname": "stork79.dropbox.com",
        "rrtype": "A",
        "tx_id": 0
    }
}


In [7]:
df = pd.json_normalize(data=only_dns)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [8]:
A_type = df[df['dns.rrtype'] == 'A']
print(len(A_type))
A_type.head()

2849


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN


In [9]:
A_type['dns.rrname'].unique()

array(['api.wunderground.com', 'stork79.dropbox.com',
       'hpca-tier2.office.aol.com.ad.aol.aoltw.net',
       'safebrowsing.clients.google.com.home', 'fxfeeds.mozilla.com',
       'www.metasploit.com', 'aolmtcmxm03.office.aol.com',
       'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm02.office.aol.com', 'hpca-tier2.office.aol.com',
       'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm04.office.aol.com', 'safebrowsing.clients.google.com',
       'wpad.home', 'safebrowsing.clients.google.com.stayonline.net',
       'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net',
       'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com',
       'AOLDTCMA04.office.aol.com', '192.168.22.110phpmyadmin',
       'secure.informaction.com', 'secure.informaction.com.localdomain',
       'safebrowsing.clients.google.com.localdomain', 'ueip.vmware.com',
       'en-us.fxfeeds.mozilla.com',
       '192.168.22.110phpmyadmin.localdomain', 'time.windows.com',
       'softwareupdate.vmwar

In [65]:
from flare.data_science.features import domain_tld_extract


df = pd.DataFrame(np.array([domain_tld_extract(x) for x in A_type['dns.rrname'].unique()]), columns=['domain'])
df.head()

clean_df = df.copy(deep=True)

## Parte 2 - Data Science

In [66]:
from caracteristicasDerivadas import H_entropy, proporcionVocalesConsonantes, posicionPrimerDigito

In [67]:
df['longitud'] = df['domain'].str.len()
df['digitos'] = df['domain'].str.count('[0-9]')
df['entropia'] = df['domain'].apply(H_entropy)
df['proporcionVocalesConsonantes'] = df['domain'].apply(proporcionVocalesConsonantes)
df['posicionPrimerDigito'] = df['domain'].apply(posicionPrimerDigito)

df.drop(['domain'], axis=1, inplace=True)

In [68]:
with open('classifier.pickle', 'rb') as handle:
    clf = pickle.load(handle)

In [69]:
preds = clf.predict(df)

In [70]:
clean_df['pred'] = preds
clean_df

,domain,pred
0,wunderground.com,0
1,dropbox.com,0
2,aoltw.net,0
3,safebrowsing.clients.google.com.home,1
4,mozilla.com,0
...,...,...
172,ntp.ubuntu.com.localdomain,0
173,microsoft.com,1
174,verisign.com,0
175,192.168.22.201:.stayonline.net,1


## Parte 3 - Dominio de experto